In [2]:
from transformers import AutoTokenizer, GPT2LMHeadModel

tokenizer = AutoTokenizer.from_pretrained("gpt2", use_fast=True)
model = GPT2LMHeadModel.from_pretrained("gpt2").eval()

In [3]:
import torch 
if torch.cuda.is_available():
    model = model.cuda()

In [4]:
from datasets import load_dataset 

dataset = load_dataset("glue", "sst2", split="validation")

In [5]:
print(f"Device: {model.device}")
print(f"Validation samples: {len(dataset)}")

tokenizer.pad_token = tokenizer.eos_token

Device: cuda:0
Validation samples: 872


In [ ]:
for i, data in enumerate(dataset.select(range(200))):
    sentence = data['sentence']
    prompt = f"""
    Classify the text as either negative or positive. 
    Text: {sentence} 
    Sentiment: """ .strip()
    tokens = tokenizer(prompt, return_tensors="pt")
    input = tokens.input_ids.cuda()
    mask = tokens.attention_mask.cuda()
    out = model.generate(input, 
                         attention_mask=mask,
                         max_new_tokens=5,
                         pad_token_id=tokenizer.pad_token_id)
    print(tokenizer.decode(out[0]))

Classify the text as either negative or positive. 
    Text: it 's a charming and often affecting journey .  
    Sentiment: it 's a very
Classify the text as either negative or positive. 
    Text: unflinchingly bleak and desperate  
    Sentiment:     
Classify the text as either negative or positive. 
    Text: allows us to hope that nolan is poised to embark a major career as a commercial yet inventive filmmaker .  
    Sentiment:  I think it
Classify the text as either negative or positive. 
    Text: the acting , costumes , music , cinematography and sound are all astounding given the production 's austere locales .  
    Sentiment:  I am a
Classify the text as either negative or positive. 
    Text: it 's slow -- very , very slow .  
    Sentiment: it 's slow --
Classify the text as either negative or positive. 
    Text: although laced with humor and a few fanciful touches , the film is a refreshingly serious look at young women .  
    Sentiment:  I'm not
Classify the text as 

Seems like prompting isn't enough for GPT2. Let's check the perplexity of SST2 with GPT2.

In [ ]:
import math

total_loss = 0.0
for data in dataset:
    sentence = data['sentence']
    tokens = tokenizer(sentence, return_tensors="pt")
    
    input = tokens.input_ids.cuda()
    mask = tokens.attention_mask.cuda()

    with torch.no_grad():
        output = model(input, labels=tokens.input_ids)

    total_loss += output.loss.item()

perplexity = math.exp(total_loss / len(dataset))
print(f"Perplexity: {perplexity}")

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
